# PROJETO 1 - AUTOMAÇÃO DE PROCESSOS - ONE PAGE REPORT

### DESCRIÇÃO:

    One Page Report é um relatório de gestão com informações estratégicas de uma área, resumida em apenas uma página.

    O objetivo é munir o gestor de informações para facilitar a sua tomada de decisão naquele dia. É a partir desse relatório que o gestor pode planejar o dia da sua equipe.

    O One Page Report transmite informações críticas de forma sintética.

    OBS.: As informações que deverão ser inseridas em um relatório One Page, deve ser decidido pelos stackeholders da área.


### CONTEXTO:

    Uma grade rede de lojas de roupas com 25 lojas espalhadas pelo Brasil necessita automatizar indicadores para facilitar a tomada de decisão.
    
    Sua tarefa será construir um One Page Report baseado nos principais indicadores solicitados  e encaminhá-los por e-mail diáriamente de forma automatizada. 
    
    
### OBSERVAÇÕES:

    Apenas o Diretor deve receber o e-mail o resumo de todas as lojas (Ranking).
    O gestores só poderão receber os indicadores referente a loja sob sua gestão.
    Necessário criar uma pasta no servidor para cada loja onde os relatórios serão armazenados para efeito histórico.
    
    
### INDICADORES:

    Por decisão dos stackeholders os seguintes indicadores serão enviados para os Gestores:
        - Faturamento
        - Quantidade de produtos vendidos
        - Ticket Médio por venda
        - Ranking para a Diretoria            
### DADOS:

    Foram enviados pelo DBA 3 planilhas contendo as informações de e-mail dos gerentes, das vendas realizadas e das lojas.
    Por definição, serão gerados diariamente pelo DBA e o seu código fará a coleta para compor o One Page Report.
    
    OBS.: Os dados poderiam ser coletados pelo próprio código abaixo realizando um SELECT nas tabelas.
    
### EXEMPLOS:
 
<p><strong>RELATÓRIO ENVIADO PARA OS GESTORES:</strong></p>   
<img src="OnePageReportGestores.png" alt="Image" height="400" width="400" align="left">
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<p><strong>RELATÓRIO ENVIADO PARA A DIRETORIA:</strong></p>   
<img src="OnePageReportDiretoria.png" alt="Image" height="400" width="400" align="left">    

## 1 - PACOTES E BIBLIOTECAS

In [1]:
import pandas as pd
import openpyxl
from pprint import pprint
import pathlib
import smtplib
import email.message
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders
from secret import remetente, senha

## 2 - CARGA DOS DADOS

In [2]:
df_emails = pd.read_excel(r'Data\Emails.xlsx', engine='openpyxl')
df_lojas = pd.read_csv(r'Data\Lojas.csv', sep=';', encoding='latin-1')
df_vendas = pd.read_excel(r'Data\Vendas.xlsx', engine='openpyxl')

## 3 - PARÂMETROS

In [3]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000

meta_qtde_produtos_dia = 4
meta_qtde_produtos_ano = 120

meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 5000

# Definindo o dia do indicador para que ele pegue sempre a última data da base de dados df_vendas
dia_indicador = df_vendas['Data'].max()

# Definindo o caminho onde os backups serão gerados
caminho = pathlib.Path(r'BackupLojas')

## 4 - ANÁLISE DOS DADOS

In [4]:
# Verificando possíveis inconssistências nos dados de cada DataFrame
df_vendas.info()
df_lojas.info()
df_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100999 entries, 0 to 100998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[ns]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  object        
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 5.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID Loja  25 non-null     int64 
 1   Loja     25 non-null     object
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes
<class 'pandas.core.frame.DataFrame'>
R

Observamos que as bases de dados não possuem arquivos faltantes ou nulos. SEndo assim não será necessário nenhum tratamento ou transformação das bases.

In [5]:
# Unindo as bases df_loja e df_vendas para posteriormente separarmos as lojas 
df_vendas = df_vendas.merge(right=df_lojas, on='ID Loja')

In [6]:
# Criando um DataFrame com informações de vendas para cada loja
lojas_dict = {}

for loja in df_lojas['Loja']:
    lojas_dict[loja] = df_vendas.loc[df_vendas['Loja'] == loja, :]

## 5 - BACKUP DOS DADOS

In [7]:
# Criando uma lista de nomes das subpastas já existentes na pasta BackupLojas que será utilizada posteriormente para verificar
# se as subpastas existem ou não.
pastas_lojas = caminho.iterdir()
lista_pasta_lojas = [pasta.name for pasta in pastas_lojas]

In [8]:
# Verificando se as subpastas das lojas existem dentro da pasta BackupLojas. Se não existir o código abaixo criará a pasta.
for loja in lojas_dict:
    try:
        if loja not in lista_pasta_lojas:
            nova_pasta = caminho / loja
            nova_pasta.mkdir()
            
    except FileExistsError :
        pass
           
    # Definindo o nome do arquivo de backup de cada loja
    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    
    # Definindo a pasta correspondente a loja onde o arquivo será salvo
    local_arquivo = caminho / loja / nome_arquivo
    
    # Realizando o backup dos arquivos das respectivas lojas
    lojas_dict[loja].to_excel(local_arquivo, index=False)

## 6 - CALCULANDO OS INDICADORES / ENVIANDO E-MAIL PARA OS GESTORES

In [18]:
# Definindo o código para buscar os arquivos que serão anexados aos e-mails.
attachment = pathlib.Path.cwd() / caminho / loja / f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'

for loja in lojas_dict:
    vendas_loja = lojas_dict[loja] 
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador,:]
    
    # FATURAMENTO
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    
    # DIVERSIDADE DE PRODUTOS
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    # TICKET MÉDIO
    valor_venda_ano = vendas_loja.groupby(by='Código Venda').sum(numeric_only=True)
    valor_venda_dia = vendas_loja_dia.groupby(by='Código Venda').sum(numeric_only=True)

    ticket_medio_ano = valor_venda_ano['Valor Final'].mean()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    # E-MAIL PARA OS GESTORES
    # Definição das cores dos indicadores de acordo com as metas
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    
    if qtde_produtos_dia >= meta_qtde_produtos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    
    if qtde_produtos_ano >= meta_qtde_produtos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    
    if ticket_medio_dia >= meta_ticket_medio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    
    if ticket_medio_ano >= meta_ticket_medio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
        
    # Corpo do e-mail
    body = f"""
    <p>Bom dia, {df_emails.loc[df_emails['Loja'] == loja,'Gerente'].values[0]}</p>

    <p>O Resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da loja <strong>{loja}</strong> foi:</p>
    <br></br>
     <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
          </tr>
          <tr>
            <td style="font-size:12px">Faturamento</td>
            <td style="text-align: center;font-size:12px">R$ {faturamento_dia:,.2f}</td>
            <td style="text-align: center;font-size:12px">R$ {meta_faturamento_dia:,.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
          </tr>
          <tr>
            <td style="font-size:12px">Diversidade de Produtos</td>
            <td style="text-align: center;font-size:12px">{qtde_produtos_dia}</td>
            <td style="text-align: center;font-size:12px">{meta_qtde_produtos_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
          </tr>
          <tr>
            <td style="font-size:12px">Ticket Médio</td>
            <td style="text-align: center;font-size: 12px">R$ {ticket_medio_dia:,.2f}</td>
            <td style="text-align: center;font-size: 12px">R$ {meta_ticket_medio_dia:,.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
          </tr>
        </table>
        <br>
        <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
          </tr>
          <tr>
            <td style="font-size:12px">Faturamento</td>
            <td style="text-align: center;font-size:12px">R$ {faturamento_ano:,.2f}</td>
            <td style="text-align: center;font-size:12px">R$ {meta_faturamento_ano:,.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
          </tr>
          <tr>
            <td style="font-size:12px">Diversidade de Produtos</td>
            <td style="text-align: center;font-size:12px">{qtde_produtos_ano}</td>
            <td style="text-align: center;font-size:12px">{meta_qtde_produtos_ano}</td>
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
          </tr>
          <tr>
            <td style="font-size:12px">Ticket Médio</td>
            <td style="text-align: center;font-size:12px">R$ {ticket_medio_ano:,.2f}</td>
            <td style="text-align: center;font-size:12px">R$ {meta_ticket_medio_ano:,.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
          </tr>
        </table>
    <br></br>
    <p>Segue em anexo a planilha para mais detalhes.</p>
    <p>Quaisquer dúvidas estou à disposição.</p>
    <p>Att., Ricardo Paganini</p>

    """
    
    msg = MIMEMultipart() 
    msg['From'] = remetente 
    msg['To'] = destinatario
    password = senha
    msg['Subject'] = f"One Page Report - {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year}"
    body = body
    msg.attach(MIMEText(body, 'html')) 
    filename = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    anexo = pathlib.Path.cwd() / caminho / loja / f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'

    attachment = open(anexo, "rb") 
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    encoders.encode_base64(p) 
  
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p) 
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(remetente, senha) 
    text = msg.as_string() 
    s.sendmail(remetente, destinatario, text) 
    s.quit() 

## 8 - RANKING DAS LOJAS / E-MAIL PARA A DIRETORIA

In [12]:
# Ranking das lojas

# Calculando o faturamento das lojas no ano e ramkeando do maior para o menor faturamento
faturamento_lojas = df_vendas.groupby(by='Loja')[['Loja','Valor Final']].sum(numeric_only=True)
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

# Salvando o arquivo de Ranking das lojas na pasta de backup (Ranking Anual)
arquivo_ano = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_RankingAnual.xlsx'
faturamento_lojas_ano.to_excel(caminho / arquivo_ano)

# Calculando o faturamento das lojas no dia e rankeando do maior para o menor
df_vendas_dia = df_vendas.loc[df_vendas['Data'] == dia_indicador,:]
faturamento_lojas = df_vendas_dia.groupby(by='Loja')[['Loja','Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

# Salvando o arquivo de Ranking das lojas na pasta de backup (Ranking Diário)
arquivo_dia = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_RankingDia.xlsx'
faturamento_lojas_dia.to_excel(caminho / arquivo_dia)

In [17]:
# Definindo os destinatários
destinatario = 'E-MAIL DO DIRETOR'

# Corpo do e-mail
body = f"""
<p>Prezado, bom dia,</p>
<p>Conforme solicitado segue abaixo o resumo dos ranking de faturamento:
<br></br>
<br><strong>*** DIÁRIO ***</strong></br>
<br>Melhor loja: {faturamento_lojas_dia.index[0]} - R$ {faturamento_lojas_dia['Valor Final'][0]:,.2f}</br>
<br>Pior loja: {faturamento_lojas_dia.index[-1]} - R$ {faturamento_lojas_dia['Valor Final'][-1]:,.2f}</br>
<br></br>
<br><strong>*** ANUAL ***</strong></br>
<br>Melhor loja: {faturamento_lojas_ano.index[0]} - R$ {faturamento_lojas_ano['Valor Final'][0]:,.2f}</br>
<br>Pior loja: {faturamento_lojas_ano.index[-1]} - R$ {faturamento_lojas_ano['Valor Final'][0]:,.2f}</br>
<br></br>
<p>Segue em anexo os relatórios detalhados.</p>
<p></p>
<p>Quaisquer dúvidas estou à disposição.</p>
<p>Att., Ricardo Paganini</p>

"""

msg = MIMEMultipart() 
msg['From'] = remetente 
msg['To'] = destinatario 
password = senha
msg['Subject'] = f"Raking Lojas - {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year}"
body = body
msg.attach(MIMEText(body, 'html')) 
filename = f'{arquivo_dia}'
filename1 = f'{arquivo_ano}'
anexo = pathlib.Path.cwd() / caminho / f'{arquivo_dia}'
anexo1 = pathlib.Path.cwd() / caminho / f'{arquivo_ano}'

attachment = open(anexo, "rb") 
p = MIMEBase('application', 'octet-stream') 
p.set_payload((attachment).read()) 
encoders.encode_base64(p) 

p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
msg.attach(p)

attachment = open(anexo1, "rb") 
p = MIMEBase('application', 'octet-stream') 
p.set_payload((attachment).read()) 
encoders.encode_base64(p)   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename1) 
msg.attach(p)

s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login(remetente, senha) 
text = msg.as_string() 
s.sendmail(remetente, destinatario, text) 
s.quit() 

(221,
 b'2.0.0 closing connection a18-20020a0568300b9200b0068bd6cf405dsm3768468otv.1 - gsmtp')

## 9 - REQUIRIMENTS.TXT DO PROJETO

In [16]:
!pip3 freeze > requirements.txt